# Biomass

In [1]:
import pandas as pd
import re
pd.options.display.max_colwidth = 300
df = pd.read_csv("biomass.csv",usecols=['Paragraph','Question','Answer'])
print(df.shape)
df.head(2)

(9764, 3)


,Paragraph,Question,Answer
0,"In such cases, temperature and CO\n 2 will have opposite effects on photosynthesis, i.e., the higher temperature induced increase in photorespiration may exceed the beneficial effect of CO\n 2 elevation, resulting in a decline in net photosynthesis. Consequently, t...",What effects does temperature and CO2 have on photosynthesis?,"Temperature and CO2 have opposite effects on photosynthesis, with the higher temperature induced increase in photorespiration potentially overriding the beneficial effect of CO2 elevation, leading to a decrease in net photosynthesis."
1,"In such cases, temperature and CO\n 2 will have opposite effects on photosynthesis, i.e., the higher temperature induced increase in photorespiration may exceed the beneficial effect of CO\n 2 elevation, resulting in a decline in net photosynthesis. Consequently, t...",What theoretical analysis has been conducted on the dependence of photosynthesis on temperature and CO2 concentration for C3 plants?,"Kirschbaum has conducted a theoretical analysis on the dependence of photosynthesis on temperature and CO2 concentration for C3 plants, finding that at 35 °C, photosynthesis at the ambient CO2 concentration reaches only 50% of the rate at saturating CO2 concentration, whereas the corresponding v..."


In [3]:
df.shape

(9764, 3)

In [4]:
import re
def cleanbasico(data):
    data = data.replace("\n"," ").strip()
    data = re.sub(' +', ' ', data)
    return data
df['Paragraph'] = df['Paragraph'].apply(lambda x:cleanbasico(x))
df['Question'] = df['Question'].apply(lambda x:cleanbasico(x))
df['Answer'] = df['Answer'].apply(lambda x:cleanbasico(x))

In [6]:
df['lp'] = df['Paragraph'].apply(lambda x: len(x.split(" ")))
df['lq'] = df['Question'].apply(lambda x: len(x.split(" ")))
df['la'] = df['Answer'].apply(lambda x: len(x.split(" ")))

In [7]:
df.describe()

,lp,lq,la
count,9764.000000,9764.000000,9764.000000
mean,189.391540,11.656493,25.563294
std,30.895528,3.985941,13.197558
min,86.000000,3.000000,1.000000
25%,176.000000,9.000000,16.000000
50%,185.000000,11.000000,23.000000
75%,196.000000,14.000000,32.000000
max,880.000000,34.000000,125.000000


In [8]:
import plotly.express as px, numpy as np, plotly.graph_objects as go
trace1=go.Box(x=df['la'].values , boxpoints='all',name="answers")
figure=go.Figure(data=[trace1])
figure.update_layout(
    title = "Words answers", font=dict(size=24)
)
figure.update_traces(orientation='h')
figure.show()

In [9]:
trace2=go.Box(x=df['lq'].values , boxpoints='all',name='questions')
figure=go.Figure(data=[trace2])
figure.update_layout(
    title = "Words questions", font=dict(size=24)
)
figure.update_traces(orientation='h')
figure.show()

In [10]:
ndf = df.query("la>4 and la<31 and lq>4 and lq<31").reset_index()
ndf.shape

(6857, 7)

In [11]:
def readPassagesQueries(all_data):
    prompt = "<s>[INST] Generate a question and answer from the following text, only use words that exist in that text and do not hallucinate.\nText:"
    all_data['QnA'] = "Question: " + all_data['Question'].apply(lambda x : " ".join(x.split())) + "\nAnswer: " +  all_data['Answer'].apply(lambda x:" ".join(x.split()))
    all_data['text'] = prompt + all_data['Paragraph'].apply(lambda x : " ".join(x.split())) + '[/INST] ' + all_data['QnA'] + "</s>"
    all_data['is_error'] = 0
    all_data.drop(columns=['Question','Answer', 'lp', 'lq', 'la'],inplace=True)
    return all_data
pd_test = readPassagesQueries(ndf)
pd_test.head()

,index,Paragraph,QnA,text,is_error
0,2,"Nevertheless, based on our experience in growing white birch seedlings in greenhouses, it appears that the species is capable of acclimating to continuous warming to more than 40 °C in the early afternoon on sunny summer days (see Figure 1). In this current study, we test the hypothesis that CO ...",Question: What is the species capable of acclimating to?\nAnswer: The species is capable of acclimating to continuous warming to more than 40 °C in the early afternoon on sunny summer days.,"<s>[INST] Generate a question and answer from the following text, only use words that exist in that text and do not hallucinate.\nText:Nevertheless, based on our experience in growing white birch seedlings in greenhouses, it appears that the species is capable of acclimating to continuous warmin...",0
1,3,"Nevertheless, based on our experience in growing white birch seedlings in greenhouses, it appears that the species is capable of acclimating to continuous warming to more than 40 °C in the early afternoon on sunny summer days (see Figure 1). In this current study, we test the hypothesis that CO ...",Question: What was the growing medium used?\nAnswer: The growing medium was a mixture of peat moss and vermiculite (1:1 (v/v)).,"<s>[INST] Generate a question and answer from the following text, only use words that exist in that text and do not hallucinate.\nText:Nevertheless, based on our experience in growing white birch seedlings in greenhouses, it appears that the species is capable of acclimating to continuous warmin...",0
2,6,"The environmental conditions in the broad-leaf chamber were controlled automatically. The environmental conditions for measuring the PNC i (C i = intercellular CO 2 concentration) curve were as follows: 26 °C and 37 °C air temperature, which were close to the highest temperatures in the early an...","Question: What environmental conditions were used for the PCI curve measurement?\nAnswer: The environmental conditions for measuring the PCI curve were 26 °C and 37 °C air temperature, 800 MOL m-2s-1 PAR and 50% relative humidity.","<s>[INST] Generate a question and answer from the following text, only use words that exist in that text and do not hallucinate.\nText:The environmental conditions in the broad-leaf chamber were controlled automatically. The environmental conditions for measuring the PNC i (C i = intercellular C...",0
3,7,Three seedlings were selected randomly from each treatment combination for the measurement. The measurement was taken on the top 5th mature leaf. All the in situ measurements were made between 9:00 and 11:30 AM with the seedlings in their original positions and conditions of the treatments. The ...,"Question: What instrument was used to measure the chlorophyll fluorescence?\nAnswer: FMS-2 portable pulse-modulated fluorometer (Hanseatic Instruments Ltd. Norfolk, UK) was used to measure the chlorophyll fluorescence.","<s>[INST] Generate a question and answer from the following text, only use words that exist in that text and do not hallucinate.\nText:Three seedlings were selected randomly from each treatment combination for the measurement. The measurement was taken on the top 5th mature leaf. All the in situ...",0
4,9,"Both gas exchange and chlorophyll fluorescence were measured after 5 months of the treatments. The apparent rate of total electron transport (J T) and its partitioning between carboxylation (J c) and oxygenation (J o) were calculated based on the methods of Farquhar et al. 26, Gent yet al. 33 an...",Question: What is the effect of Temperature and CO2 on PN?\nAnswer: There was a significant (P<0.01) interactive effect of temperature and CO2 on PN.,"<s>[INST] Generate a question and answer from the following text, only use words that exist in that text and do not hallucinate.\nText:Both gas exchange and chlorophyll fluorescence were measured after 5 months of the treatments. The apparent rate of total electron transport (J T) and its partit..

In [12]:
pd_test.to_csv("finewithbiomass.csv",index=False)